# Assignment 1 - Word2Vec
1.  Try a real corpus (instead of banana apple, try something real... on the internet....) - not so big!  

Just you have a good taste of real stuff....like 50 documents, each have 50 words....

2. Try window size of 2

3. Try CBOW (instead of skipgrams)

4. Compare normal version of skipgrams vs. negative sampling version of skipgrams in terms of time (using real corpus)

In [1]:
import re
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import spacy
import pandas as pd
spacy.__version__

'3.4.2'

## 1 Load Data
A real corpus

In [2]:
nlp = spacy.load('./en_core_web_sm/')
text = open('./dataset/alchemist.txt',mode='r') #change later
df = pd.DataFrame(text.readlines())

### Clean Data

In [3]:
def clean_data(df_col):
    corpus = []
    for item in df_col:
        item = re.sub('[^A-Za-z0-9]+', ' ', str(item)) # remove special characters
        item = item.lower() # lower all characters
        item = item.split() # split data
        corpus.append(' '.join(str(x) for x in item))
    return corpus

In [4]:
#1. tokenize
#data cleaned
corpus = clean_data(df[0])
#data tokenized
corpus_tokenized = [sent.split(" ") for sent in corpus]

In [5]:
#2. numericalize (vocab)
#2.1 get all the unique words
#we want to flatten unit (basically merge all list)
flatten = lambda l: [item for sublist in l for item in sublist]
vocabs = list(set(flatten(corpus_tokenized)))

#2.2 assign id to all these vocabs
word2index = {v: idx for idx, v in enumerate(vocabs)}

#adding unknown word
vocabs.append('<UNK>')
word2index['<UNK>'] = len(vocabs) - 1

voc_size = len(vocabs)
voc_size

1233

### Testing Skipgram and CBOW

In [24]:
skipgrams = []
cbow = []
for sent in corpus_tokenized:
    #for each sent ('apple', 'banana', 'fruit')
    for i in range(2,len(sent)-2): #start from 2 to second last
        center_word = sent[i]
        outside_word = [sent[i-2],sent[i-1],sent[i+1],sent[i+2]] #window size = 2
        cbow.append((outside_word, center_word))
        for o in outside_word: 
            skipgrams.append((center_word,o))

In [25]:
skipgrams[:5]

[('crowning', 'high'),
 ('crowning', 'up'),
 ('crowning', 'the'),
 ('crowning', 'grassy'),
 ('the', 'up')]

In [26]:
cbow[:4]

[(['high', 'up', 'the', 'grassy'], 'crowning'),
 (['up', 'crowning', 'grassy', 'summit'], 'the'),
 (['crowning', 'the', 'summit', 'of'], 'grassy'),
 (['the', 'grassy', 'of', 'a'], 'summit')]

### Set random_batch

In [9]:
def random_batch(batch_size, corpus,window_size=1):
    skipgrams = []
    #for each corpus
    for sent in corpus_tokenized:
        #for each sent ('apple', 'banana', 'fruit')
        for i in range(window_size,len(sent)-window_size): #start from 2 to second last
            # print(sent[i])
            center_word = word2index[sent[i]]
            for j in range(window_size):
                outside_word = [word2index[sent[i-j]],word2index[sent[i+j]]] #window_size adjustable
                #here we want to create (banana, apple), (banana, fruit) append to some list
                for o in outside_word:
                    skipgrams.append([center_word,o])
    #only get a batch, mot the entire lsit
    random_index = np.random.choice(range(len(skipgrams)),batch_size,replace=False)
    
    #appending some list of inputs and labels
    random_inputs, random_labels = [] , []
    for index in random_index:
        random_inputs.append([skipgrams[index][0]]) #center words, this will be as shape of (1,) -> (1,1) for modeling
        random_labels.append([skipgrams[index][1]])

    return np.array(random_inputs),np.array(random_labels)

In [10]:
#preparing all_vocabs
batch_size = 2

def prepare_seqeunce(seq, word2index):
    #map(fucntion, list of something)
    #map will look at each of element in this list, and apply this function
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index["<UNK>"], seq))
    return torch.LongTensor(idxs)

all_vocabs = prepare_seqeunce(list(vocabs),word2index).expand(batch_size, voc_size)
all_vocabs.shape

torch.Size([2, 1233])

## 2.1 Continuous Bag-of-Words (CBOW)

### CBOW Model

In [ ]:
class CBOW(nn.Module):
    def __init__(self,voc_size, emb_size):
        super(CBOW,self).__init__()
        self.embedding_center_word = nn.Embedding(voc_size, emb_size) #is a lookup table mapping all ids in voc_size, into some vector of size emb_size
        self.embedding_outside_word = nn.Embedding(voc_size, emb_size)
    
    def forward(self, center_word, outside_word, all_vocabs):
        pass

In [ ]:
batch_size = 2 #why? no reason
emb_size = 2 #why? no reason; usually 50,100, 300 but 2 so we can plot (50 can also plot, but need PCA)
model = CBOW(voc_size,emb_size)

criterion = nn.CrossEntropyLoss() #-log
optimizer = optim.Adam(model.parameters(), lr=0.001)

### training

In [ ]:
import time
num_epochs = 5000
#for epoch
start = time.time()
for epoch in range(num_epochs):
    
    #get random batch
    input_batch, label_batch = random_batch(batch_size,corpus)
    input_batch = torch.LongTensor(input_batch)
    label_batch = torch.LongTensor(label_batch)

    # print(input_batch.shape,label_batch.shape,all_vocabs.shape)
    # break

    #loss = model
    loss = model(input_batch,label_batch,all_vocabs)
    #backpropagate
    loss.backward()
    #update alpha
    optimizer.step()

    end = time.time()

    #print epoch loss
    if (epoch + 1) % 1000 == 0:
        print(f"Epoch: {epoch + 1} | cost: {loss:.6f} | time: {end-start}")

# Compare normal version of skipgrams vs. negative sampling version of skipgrams in terms of time (using real corpus)

## 2.2 Skipgram

### Skipgram Model

In [11]:
class Skipgram(nn.Module):
    def __init__(self,voc_size, emb_size):
        super(Skipgram,self).__init__()
        self.embedding_center_word = nn.Embedding(voc_size, emb_size) #is a lookup table mapping all ids in voc_size, into some vector of size emb_size
        self.embedding_outside_word = nn.Embedding(voc_size, emb_size)
    
    def forward(self, center_word, outside_word, all_vocabs):
        #center_word, outside_word: (batch_size,1)
        #all_vocabs : (batch_size, voc_size)
        #convert them into embedding
        center_word_embed = self.embedding_center_word(center_word)     #v_c (batch_size,1, emb_size)
        outside_word_embed = self.embedding_outside_word(outside_word)  #u_o (batch_size,1, emb_size)
        all_vocabs_embed = self.embedding_outside_word(all_vocabs)      #u_w (batch_size,voc_size, emb_size)
        # print(center_word_embed.shape,outside_word_embed.shape,all_vocabs_embed.shape)
        #bmm is basically @ or .dot but across batches (ie., ignore the batch dimension)
        top_term = outside_word_embed.bmm(center_word_embed.transpose(1,2)).squeeze(2)
        #(batch_size,1, emb_size) @ (batch_size, emb_size, 1) = (batch_size, 1, 1) ===> (batch_size, 1)
        top_term_exp = torch.exp(top_term) #exp(uo vc)
        #(batch_size, 1)
        lower_term = all_vocabs_embed.bmm(center_word_embed.transpose(1,2)).squeeze(2)
        #(batch_size, voc_size, emb_size) @ (batch_size, emb_size, 1) = (batch_size, voc_size, 1) ===> (batch_size, voc_size)
        lower_term_sum = torch.sum(torch.exp(lower_term)) #sum exp(uw, vc)
        #(batch_size, 1)
        loss_fn = -torch.mean(torch.log(top_term_exp/lower_term_sum))
        #(batc_size,1) / (batch_size,1) ==mena==> scalar
        return loss_fn

In [13]:
batch_size = 2 #why? no reason
emb_size = 2 #why? no reason; usually 50,100, 300 but 2 so we can plot (50 can also plot, but need PCA)
model = Skipgram(voc_size,emb_size)

criterion = nn.CrossEntropyLoss() #-log
optimizer = optim.Adam(model.parameters(), lr=0.001)

### training

In [14]:
import time
num_epochs = 5000
#for epoch
start = time.time()
for epoch in range(num_epochs):
    
    #get random batch
    input_batch, label_batch = random_batch(batch_size,corpus)
    input_batch = torch.LongTensor(input_batch)
    label_batch = torch.LongTensor(label_batch)

    # print(input_batch.shape,label_batch.shape,all_vocabs.shape)
    # break

    #loss = model
    loss = model(input_batch,label_batch,all_vocabs)
    #backpropagate
    loss.backward()
    #update alpha
    optimizer.step()

    end = time.time()

    #print epoch loss
    if (epoch + 1) % 1000 == 0:
        print(f"Epoch: {epoch + 1} | cost: {loss:.6f} | time: {end-start}")

Epoch: 1000 | cost: 6.958669 | time: 9.76584768295288
Epoch: 2000 | cost: 4.852635 | time: 19.432329893112183
Epoch: 3000 | cost: 3.855736 | time: 28.415507316589355
Epoch: 4000 | cost: 6.150964 | time: 38.61642026901245
Epoch: 5000 | cost: 6.576195 | time: 48.29460906982422


## 2.3 Negative Sampling

In [15]:
##Unigram Distribution
z = 0.0001
#count all the occurence of vocabs
from collections import Counter

word_count = Counter(flatten(corpus_tokenized))
word_count

num_total_words = sum([c for w, c in word_count.items()])
num_total_words

unigram_table = []

for v in vocabs:
    uw = word_count[v]/num_total_words
    uw_alpha = uw ** 0.75
    uw_alpha_dividebyz = int(uw_alpha/z)
    # print('Vocab :',v)
    # print('distribution :', uw_alpha_dividebyz)
    unigram_table.extend([v] * uw_alpha_dividebyz)

# Counter(unigram_table)

In [16]:
def prepare_sequence(seq, word2index):
    #map(fucntion, list of something)
    #map will look at each of element in this list, and apply this function
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index["<UNK>"], seq))
    return torch.LongTensor(idxs)
    
import random
#you don't want to pick samples = targets, basically negative samples
#k = number of negative samples - how many? they found 10 is the best
#will be run during training
#after random_batch, 
def negative_sampling(targets, unigram_table, k):
    #targets is already in id.....
    #but the unigram_table is in word....
    #1. get the batch size of this targets
    batch_size = targets.shape[0]
    neg_samples = []
    #2. for each batch
    for i in range(batch_size):
        #randomly pick k negative words from unigram_table
        target_index = targets[i].item()  #looping each of the batch....
        nsample = []
        while len(nsample) < k:
            neg = random.choice(unigram_table)
            #if this word == target, skip this word
            if word2index[neg] == target_index:
                continue
            nsample.append(neg)
        #append this word to some list
        neg_samples.append(prepare_sequence(nsample, word2index).reshape(1, -1))  #tensor[], tensor[]
    return torch.cat(neg_samples)  #tensor[[], []]

In [17]:
num_neg = 2 # in the real code, we gonna use 10 (like in the paper)
neg_samples = negative_sampling(label_batch, unigram_table, num_neg)
# neg_samples[0].shape
neg_samples.shape 

torch.Size([2, 2])

### Negative Sampling Model

In [18]:
#the model will accept three vectors - u_o, v_c, u_k
#u_o - vectos for outside words
#v_C - vector for center word
#u_k - vectors for negative word

class SkipgramNeg(nn.Module):
    def __init__(self,voc_size, emb_size):
        super(SkipgramNeg,self).__init__()
        self.embedding_center_word = nn.Embedding(voc_size, emb_size)
        self.embedding_outside_word = nn.Embedding(voc_size, emb_size)
        self.logsigmoid = nn.LogSigmoid()

    def forward(self, center_words, outside_words, negative_words):
        #center_words, outside_words  (batch_size,1)
        #negative_words (batch_size,k) 
        center_embed    = self.embedding_center_word(center_words)      #(batch_size,1, emb_size)
        outside_embed   = self.embedding_outside_word(outside_words)   #(batch_size,1, emb_size)
        neg_embed       = self.embedding_outside_word(negative_words)      #(batch_size,k, emb_size)
        
        uovc            = outside_embed.bmm(center_embed.transpose(1,2)).squeeze(2)
        ukvc            = -neg_embed.bmm(center_embed.transpose(1,2)).squeeze(2)
        ukvc_sum        =  torch.sum(ukvc, 1).view(-1, 1) #(batch_size, 1)
        loss = self.logsigmoid(uovc) + self.logsigmoid(ukvc_sum) #(batch_size,1)+(batch_size,1)
        
        return -torch.mean(loss) #scalar, loss should be scalar, to call backward()

In [19]:
voc_size = len(vocabs)
voc_size

batch_size = 2 #why? no reason
emb_size = 2 #why? no reason; usually 50,100, 300 but 2 so we can plot (50 can also plot, but need PCA)
model = SkipgramNeg(voc_size,emb_size)

criterion = nn.CrossEntropyLoss() #-log
optimizer = optim.Adam(model.parameters(), lr=0.001)

### training

In [23]:
import time

# Training
num_epochs = 5000
start = time.time()
for epoch in range(num_epochs):
    
    input_batch, target_batch = random_batch(batch_size, corpus)
    
    #input_batch: [batch_size, 1]
    input_batch = torch.LongTensor(input_batch)
    #target_batch: [batch_size, 1]
    target_batch = torch.LongTensor(target_batch)
    
    #negs_batch:   [batch_size, num_neg]
    negs_batch = negative_sampling(target_batch, unigram_table, num_neg)
    
    optimizer.zero_grad()
        
    loss = model(input_batch, target_batch, negs_batch)
    
    end = time.time()
    
    loss.backward()
    optimizer.step()
    
    #print loss
    if (epoch + 1) % 1000 == 0:
        print(f"Epoch: {epoch + 1} | cost: {loss:.6f} | time: {end - start :.2f}")

Epoch: 1000 | cost: 1.905224 | time: 8.41
Epoch: 2000 | cost: 0.764615 | time: 17.18
Epoch: 3000 | cost: 0.995515 | time: 25.71
Epoch: 4000 | cost: 0.519658 | time: 34.71
Epoch: 5000 | cost: 1.362329 | time: 43.98


# Appendix

## Implement with any window_size

In [21]:
def dataBags(corpus_tokenized):
    # data - [(context), target]
    data = []
    for sent in corpus_tokenized:
        for i in range(2, len(sent) - 2):
            context = [sent[i - 2], sent[i - 1], sent[i + 1], sent[i + 2]]
            target = sent[i]
            data.append((context, target))

    return data

cbow = dataBags(corpus_tokenized)
cbow[:5]

[(['high', 'up', 'the', 'grassy'], 'crowning'),
 (['up', 'crowning', 'grassy', 'summit'], 'the'),
 (['crowning', 'the', 'summit', 'of'], 'grassy'),
 (['the', 'grassy', 'of', 'a'], 'summit'),
 (['grassy', 'summit', 'a', 'swelling'], 'of')]

## Example to import db file

In [22]:
import pandas as pd
import sqlite3
def ReadSQL(filename):
    connection = sqlite3.connect(filename)
    data = pd.read_sql("SELECT * from city_table",connection)
    return data

ReadSQL('./dataset/city.db')
#It doesn't use it

,City,Country,Population
0,athens,greece,1368
1,bangkok,thailand,1178
2,barcelona,spain,1280
3,berlin,east_germany,3481
4,birmingham,united_kingdom,1112
...,...,...,...
66,tokyo,japan,8535
67,toronto,canada,668
68,vienna,austria,1766
69,warsaw,poland,965
